In [1]:
#setting libraries
import os
import re
import datetime

Data overview

In [ ]:
#reading data and separating data into test and bench files
tests_files=[a for a in os.listdir('./data.nogit/') if 'test' in a]
bench_files=[a for a in os.listdir('./data.nogit/') if 'bench' in a]
# os.getcwd()
#overview of the test files
bits={} #key:file name value:number of bits

for a in tests_files:
    total_bits=0
    n_input=0
    n_outpus=0
    with open(f'./data.nogit/{a}','r') as file:
        while True:
            line=file.readline().strip()
            if not(line):
                break
            else: #there's available data to read
                total_bits=max(total_bits,len(line))
                n_outpus=line.count('-')
                n_input=len(line)-n_outpus
    bits[a]={"total_bits":total_bits,"INPUT":n_input,"OUTPUT":n_outpus}
    file.close()

for a,circ_struc in bits.items():
    print(f'file: {a}, total bits to work: {circ_struc}')
def extract_number_from_string(string:str):
    '''
    :param str string: the string of values to evaluate
    input string to extract the integer value
    '''
    print(string)
    aux=re.findall(r'(\d+) \w+',string)
    print(aux)
    return int(aux[0])
#overview of bench files

n_vales={} #key:file_name value:{key: input/output/gates value:read value from file}
exp_values={} #key: input/output/gates value:expected number
for a in bench_files:
    n_vales[a]={}
    exp_values[a]={}
    aux=0
    print(f'read file: {a}')
    with open(f'./data.nogit/{a}','r') as file:
        while True:
            line=file.readline().strip()
            if aux>3:
                break
            if not(line):
                aux+=1           
                continue
            else:
                if line[0]=='#': #it show the information about the file
                    try:
                        exp_values[a][line.split(' ')[2].upper()]=extract_number_from_string(line.lower())
                    except:
                        continue
                else:
                    gate_str="".join(filter(str.isalpha,line))
                    if gate_str not in n_vales[a]:
                        n_vales[a][gate_str]=0
                    n_vales[a][gate_str]+=1
    file.close()

gates=[] #getting the names of the gates used in the bench files
for a,circ_struc in n_vales.items():
    print (a,circ_struc)
    for hierarchy  in circ_struc.keys():
        if hierarchy not in ['INPUT','OUTPUT']:
            if hierarchy not in gates:
                gates.append(hierarchy)

print(gates)

Code for running tests

In [2]:
#setting time to Japan time (UTC+9)
jpTime=datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
#when testing started
t0=jpTime.now()
def log(msg:str):
    '''
    :param str msg: the message to appear in the log
    '''
    delta=jpTime.now()-t0 #show the running time
    print(f'{delta}...{msg}')
#logic gate tryout
def logic_gate(gate:str, inputs:list):
    '''
    defining the logic gate functioning

    Params:
        gate(str): 'NAND', 'AND', 'OR', 'NOT', 'NOR', 'BUFF', 'XOR' name of gates in bench files
        inputs(list): defines the inputs for the gate [A,B,C...], the number of elements in the function will determine the amount of pins for the logic gate.EXCEPT for NOT gates it will take ONLY the first element

    Returns:
        output(bool): a single logic value to assign to the corresponding 
    '''
    #verifying that the input list is just integers 0/1
    aux,inputs=inputs,[]
    inputs=[int(x) for x in aux]
    if gate=='NAND':
        holder=1
        [holder:=holder&x for x in inputs]
        return 0 if holder else 1
    elif gate=="AND":
        holder=1
        [holder:=holder&x for x in inputs]
        return 1 if holder else 0
    elif gate=="OR":
        holder =0
        [holder:=holder|x for x in inputs]
        return 1 if holder else 0
    elif gate=="NOT":
        return 0 if inputs[0] else 1
    elif gate=="NOR":
        holder =0
        [holder:=holder|x for x in inputs]
        return 0 if holder else 1
    elif gate=="BUFF":
        return 1 if inputs[0] else 0
    elif gate=="XOR":
        holder=0
        [holder:=holder^x for x in inputs]
        return 0 if holder else 1

In [55]:
def truth_table(primary_inputs:list, primary_outputs:list,circ_struc:list):
    '''
    Params:
        primary_inputs(list): A list of the primary input signals index
        primary_outputs(list): A list of the primary output signals index
        circ_struct(list): A list with the relationships between inputs and outputs
    Returns:
        fault_free_circuit(dict): key:binary input for primary inputs value:fault-free output
    '''
    log("Beginning truth table generation")
    b={}
    fault_free_circuit=[]
    for a in range(2**len(primary_inputs)):
        aux=0
        c=bin(a)[2:].zfill(len(primary_inputs))
        while aux<len(primary_inputs):
            b[primary_inputs[aux]]=int(c[aux])
            aux+=1
        normal_output=circuit_code(b,circ_struc)
        bench_outputs=[normal_output[x] for x in primary_outputs]
        # print(bin(a)[2:].zfill(3),bench_outputs)
        fault_free_circuit.append(bench_outputs)
    log("Finished generating truth table")
    return fault_free_circuit

In [ ]:
#reading the bench file to assign the input, output and intermediate connections.
#read only once (the rutine is the same for ALL the data)
def read_bench(bench_file:str):
    """_summary_

    Args
    -------
        bench_file : str
            the path of the bench file to evaluate

    Returns
    -------
        circ_struct: list
            relationship input, gate and output

        signal_hierarchy: dict
            signals map of the circuit
        
        truth table: dict
            ONLY if the circuit has 8 or less inputs, the truth table is generated
    """      
    aux=0
    circ_struc=[] #circuit structure
    signal_hierarchy={} #key: level #value[list]: list of inputs. All signals are considered input
    signal_hierarchy[0]=[]
    fanout_stems=0
    n_inv=0
    with open(bench_file) as f:
        while True:
            a=f.readline().strip()
            if aux>3:
                break
            if not(a):
                aux+=1
                continue
            else:
                if "#" in a:
                    continue
                else:
                    if 'INPUT' in a or 'OUTPUT' in a: #connections for primary input and primary output
                        
                        b=a.split('(')[1][:-1]
                        if 'INPUT' in a:
                            signal_hierarchy[0].append(b)
                    else: # signal = gate (input signal)
                        circ_struc.append(a)
                        y='0'
                        
                        if 'NOT' in a: #getting the number of inverters
                            n_inv+=1

                        for z in a.split(' = ')[1].split('(')[1][:-1].split(', '):
                            y=max(y,z)
                            for x in signal_hierarchy.values():
                                print(z, x)
                                if z in x:
                                    fanout_stems+=1
                        ind=0
                        #evaluates in string format
                        while ind<len(signal_hierarchy):
                            if y not in signal_hierarchy[ind]:
                                ind+=1
                            else:
                                break
                        ind+=1
                        if ind not in signal_hierarchy:
                            signal_hierarchy[ind]=[]
                        signal_hierarchy[ind].append(a.split(' = ')[0])
    f.close()
    log (f"Finished reading bench file: {bench_file}")
    
    aux=0
    for z in signal_hierarchy.values():
        aux+=len(z)
    log (f'There area {aux*2} possible faulty circuits under the single-fault assumption')
    print(f'Number of POs: {len(signal_hierarchy[len(signal_hierarchy)-1])}')
    print(f'Number of fanout stems: {fanout_stems}')
    # print(f'Number of gate inputs {}')
    print(f'Number of inverters: {n_inv}')


    if len(signal_hierarchy[0])<8: #it was chosen 8, in order to use an 8 switch input from wokwi
        log(f'circuit is small enough to perform a functional testing, generating truth table')
        z=truth_table(signal_hierarchy[0],signal_hierarchy[len(signal_hierarchy)-1],circ_struc)
        return circ_struc, signal_hierarchy,z
    else:
        log(f'circuit has over 10  inputs, too large to perfom a functional testing')
        return circ_struc, signal_hierarchy 

In [92]:
#circuit code
def circuit_code(signal_line:dict, circ_struc:list, stuck_at:dict={}):
    """This runs the circuit in fault-free mode and stuck-at fault

    Args:
        signal_line (dict): Imports what the signal logic value must be
        circ_struc (list): Which logic gate and the relationship between inputs and outputs
        stuck_at (dict): If provided, these values are flipped to whichever value is in this dict #key=signal index value=logic value to be stuck at
    Returns:
        signal_line (dict): returns the updated signal logic value for any situation (fault-free or stuck-at fault)
    """   
    for a in circ_struc:
        output_signal=a.split(' = ')[0]
        gate=a.split(' = ')[1].split('(')[0]
        input_signal_index=a.split(' = ')[1].split('(')[1][:-1]

        input_signals=[]
        for x in input_signal_index.split(', '):
            if x in stuck_at:
                 input_signals.append(stuck_at[x.strip()])
            else:
                input_signals.append(signal_line[x.strip()])

        signal_line[output_signal]=logic_gate(gate,input_signals)
        if stuck_at and (output_signal in stuck_at.keys()):
                signal_line[output_signal]=stuck_at[output_signal]
    return signal_line

In [6]:
#reading the test file for data and testing
#this script produces the "correct" behaviour
def read_test(test_file: str,circ_struc: list,signals:dict):
    """reads the test vector form the test file and generates the corresponding output

    Args:
        test_file (str): file with test vectors
        circ_struc (list): It contains the information of the logic gates and it inputs and outputs signals
        signals (dict): It has the hierarchy of all signal where [0]= primary inputs and [len(signals)-1]= primary outputsca

    Returns:
        _type_: _description_
    """    
    primary_input=signals[0]
    primary_output=signals[len(signals)-1]
    signal_line={}
    with open(test_file) as f:
        while True:
            line=f.readline().strip()
            if not(line):
                break
            else: #I assume the data is organized in the bench stated order
                #assigning the test data to the corresponding signals
                for a in range(len(primary_input)):
                    try:
                        signal_line[primary_input[a]]=int(line[a])
                    except:
                        continue
                
                #executing the testing from the bench file (running the circuit)
                #structure= output signal= logic gate (input signal)
                signal_line=circuit_code(signal_line,circ_struc)
                
                circuit_output=[signal_line[x] for x in primary_output]
                print(f'Data input: {line[:line.index('-')]}, \tData output: {circuit_output}')
    f.close()
    log(f"Finished running test file {test_file}")
    return circuit_output

Code to draw the circuit in the Wokwi

In [7]:
a,b,c=read_bench('./data.nogit/c1.bench')
# d=read_test('./data.nogit/c17.tests',a,b)

0:00:10.927428...Finished reading bench file: ./data.nogit/c1.bench
0:00:10.928384...There area 18 possible faulty circuits under the single-fault assumption
0:00:10.928410...circuit is small enough to perform a functional testing, generating truth table
0:00:10.928431...Beginning truth table generation
0:00:10.928638...Finished generating truth table


In [ ]:
#logic gates codes to draw circuits in wokwi
def logic_gate_diagram(gate:str,number:int,top:int,left:int):
    '''
    Params
    -------
        gate(str): the logic gate desired
        number(int): the number for id
        top(int): top value for wokwi reference system
        left(int): left value for wokwi reference system
    Returns
    -------
        part_json(str): the corresponding json to draw the circuit on Wokwi
        output_json(str): the correspondin json to indicate the ouput\
    '''
    #NAND gate
    if gate=='NAND':
        type="wokwi-gate-nand-2"
    if gate=='AND':
        type="wokwi-gate-and-2"
    if gate=='NOR':
        type= "wokwi-gate-nor-2"
    if gate=='OR':
        type="wokwi-gate-or-2"
    if gate=='X0R':
        type="wokwi-gate-xor-2"
    if gate=='NOT':
        type="wokwi-gate-not"
    if gate=='BUFF':
        type="wokwi-gate-buffer"
    return f'"type":"{type}", "id":"{gate.lower()}{str(number)}","top":{top},"left":{left}'

In [ ]:
#logic gates connections
#[start, finish, color, ["h","v"]]
[ "sw1:1b", "nand1:A", "black", [ "h9.3", "v-9.6" ] ],

#signal number
{ "type": "wokwi-text", "id": "text16", "top": -384, "left": 0, "attrs": { "text": "16" } },

#spacing between logic gates
#same left level gates
#gate and output signal text delta_top=0 delta_left=100



In [ ]:
def gate_left_top(circ_struc:list,hierarchy:dict):

    rel_pos={} #key: signal #value[left,top]
    aux=0
    #set left position
    for y,z in hierarchy.items():
        for x in z:
            rel_pos[x]=[y]
            if y==0:
                rel_pos[x].append(aux)
                aux+=1

    for y in circ_struc:
        t=y.split(' = ')[0]
        u=y.split(' = ')[1].split('(')[1][:-1].split(', ')
        w=0
        for x in u:
            w+=rel_pos[x][1]
        rel_pos[t].append(w/len(u))
    return rel_pos

In [ ]:
rel_pos=gate_left_top(b,c)

In [ ]:
for y,z in rel_pos.items():
    print(y,z)

In [ ]:
for d,e in enumerate(b):
    #determine logic gate
    gate=e.split(' = ')[1].split('(')[0]
    
    #determine the output
    g_out=e.split(' = ')[0]
    print("{",logic_gate_diagram(gate,d,rel_pos[g_out][1]*(100),rel_pos[g_out][0]*120),"},")
print('{ "type": "wokwi-dip-switch-8", "id": "sw1", "top": 0, "left": 0, "rotate": 90, "attrs": {} }')#adding a switch for the ipnuts


In [ ]:
wokwi_diagram_json={
    "version":1,
    "author":"",
    "editor":"wokwi",
    "parts":[], #logic gates and text
    "connnections":[],#wires between logic gates
    "dependencies":{}
}

Stuck-at fault code

In [110]:
a='c1.bench'
circuit,hierarchy,signal=read_bench(f'./data.nogit/{a}')
# fault_free=truth_table(pi,po,circuit)

b ['a', 'b', 'c']
b ['a', 'b', 'c']
b ['d']
e ['a', 'b', 'c']
e ['d', 'e']
f ['a', 'b', 'c']
f ['d', 'e']
f ['f']
c ['a', 'b', 'c']
c ['d', 'e']
c ['f']
a ['a', 'b', 'c']
a ['d', 'e']
a ['f']
a ['h']
d ['a', 'b', 'c']
d ['d', 'e']
d ['f']
d ['h']
g ['a', 'b', 'c']
g ['d', 'e']
g ['f', 'g']
g ['h']
h ['a', 'b', 'c']
h ['d', 'e']
h ['f', 'g']
h ['h']
2:19:09.175885...Finished reading bench file: ./data.nogit/c1.bench
2:19:09.175917...There area 18 possible faulty circuits under the single-fault assumption
Number of POs: 1
Number of fanout stems: 9
Number of inverters: 1
2:19:09.175958...circuit is small enough to perform a functional testing, generating truth table
2:19:09.175969...Beginning truth table generation
2:19:09.176076...Finished generating truth table


Fault equivalency simulations for every logic gate

For any gate try SA0 and SA1 at both input and output

In [ ]:
#for each logic gate testing fault colapsing
def brute_SA_faults_logic_gates_colapsing(gate:str,inputs:str|int):
    '''
    Params:
        gate(str): the logic gate to evaluate
        inputs(str|int): is the number of inputs for the gate str:'0000' int:2
    Returns:
    '''

    # inputs=2
    # gate='NAND'
    if isinstance(inputs,int):
        lenght=inputs
    elif isinstance(inputs,str):
        lenght=len(inputs)
    else:
        # return "the provided input is not in a valid format"
        print("the provided input is not in a valid format")

    test='SA0'
    failure=''
    signal=['A','B','Output']
    #SA for the inputs
    for a in range(2**lenght+2):
        for b in range(2**lenght):
            pi_input=bin(b)[2:].zfill(lenght)
            po=logic_gate(gate,list(pi_input))
            sa_input=pi_input[0:(a//2)]+str(int(test[-1]))+pi_input[(a//2)+1:]
            if a<2**lenght:
                sa_output=logic_gate(gate,list(sa_input))
            else:
                sa_input='xx'
                sa_output=int(test[-1])
            
            if sa_output!=po:
                failure="failed"
            else:
                failure=''
            print(f'|{gate}|{pi_input}|{test}[{signal[a//2]}]|{sa_input}|{po}|{sa_output}|{failure}|')
        if test=='SA0':
            test='SA1'
        else:
            test='SA0'

In [14]:
list(circuit.values())

[['a', 'b', 'c'], ['d', 'e'], ['f', 'g'], ['h'], ['y']]

In [96]:
#for perfoming brute SA fault 
def brute_SA_single_faults(circuit:list, hierarchy:dict):
    """performs an Stuck-at 0 and Stuck-at 1 single fault for every circuit segment
    Compares the obtained result against the truth table

    Args:
        circuit (list): relationship between logic gate, input and output
        hierarchy (dict): signal map of the circuit

    Returns:
        SA_signal_line (dict): key:signal_SA0/SA1 value:[list of output]
    """    
    lenght=[]
    test_vector={}
    SA_signal_line={}
    for z in hierarchy.values():
        for y in z:
            lenght.append(y)
    #SA for the inputs
    for a in lenght:
        for test in ['SA0','SA1']:
            SA_signal_line[f'{a}_{test}']=[]
            for b in range(2**len(hierarchy[0])):
                aux=0
                c=bin(b)[2:].zfill(len(hierarchy[0]))
                while aux<len(hierarchy[0]):
                    test_vector[hierarchy[0][aux]]=int(c[aux])
                    aux+=1
                # print(test_vector,circuit,{a:int(test[-1])})
                outputs=circuit_code(test_vector,circuit,{a:int(test[-1])})
                SA_signal_line[f'{a}_{test}'].append([outputs[x] for x in hierarchy[len(hierarchy)-1]])
    return SA_signal_line

In [97]:
test_vector={}
for d in SA_signal_line.keys():
    # print(d, SA_signal_line[d])
    aux=0
    test_vector[d]=[]
    while aux<len(signal):
        # print(signal[aux],end='|')
        # print(SA_signal_line[d][aux])
        if signal[aux]!=SA_signal_line[d][aux]:
            test_vector[d].append(bin(aux)[2:].zfill(len(hierarchy[0])))
        aux+=1
for y,z in test_vector.items():
    print(y,z)

a_SA0 ['110', '111']
a_SA1 ['010', '011']
b_SA0 ['011', '110']
b_SA1 ['001', '100']
c_SA0 ['001', '101']
c_SA1 ['000', '100']
d_SA0 ['110', '111']
d_SA1 ['100']
e_SA0 ['011']
e_SA1 ['001', '101']
f_SA0 ['001', '101']
f_SA1 ['011']
g_SA0 ['110', '111']
g_SA1 ['000', '010', '011', '100']
h_SA0 ['001', '101']
h_SA1 ['000', '010', '011', '100']
y_SA0 ['001', '101', '110', '111']
y_SA1 ['000', '010', '011', '100']


In [ ]:
#making a 2 bit truth table 
aux=0
l_gates=['NAND', 'AND', 'OR', 'NOT', 'NOR', 'BUFF', 'XOR']
while aux<len(l_gates):
    print('|Logic Gate|primary input|SA type|stuck-at fault input|primary output|stuck-at output|failed?|')
    print('| :---: '*7)
    print(brute_SA_faults_logic_gates_colapsing(l_gates[aux],2))
    print('-'*30)
    aux+=1